In [1]:
# !pip install shazamio

In [2]:
1+1

2

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import pandas as pd
import requests
from tqdm import tqdm_notebook
import glob
import time

In [5]:
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
from requests.exceptions import ChunkedEncodingError

In [6]:
# Считываем данные из таблицы (CSV или Excel, в зависимости от формата)
file_path = '../../yappy_hackaton_2024_40k_small.csv'  # укажите правильный путь к файлу
df = pd.read_csv(file_path)  # если у вас Excel, используйте pd.read_excel(file_path)
df = df.reset_index()
df = df.rename(columns={'index': 'index orig'})

In [7]:
df_part = df.iloc[0:100]

In [8]:
df_part

,index orig,Unnamed: 0,link,description
0,0,0,https://cdn-st.rutubelist.ru/media/b0/e9/ef285...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m..."
1,1,1,https://cdn-st.rutubelist.ru/media/39/6c/b31bc...,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #rob...
2,2,2,https://cdn-st.rutubelist.ru/media/e9/e0/b47a9...,"#boobs , #красивыедевушки , #ass"
3,3,3,https://cdn-st.rutubelist.ru/media/87/43/b11df...,NaN
4,4,4,https://cdn-st.rutubelist.ru/media/d1/e7/642dc...,NaN
...,...,...,...,...
95,95,95,https://cdn-st.rutubelist.ru/media/6a/29/c0c86...,#красивыедевушки #танцы
96,96,96,https://cdn-st.rutubelist.ru/media/e9/88/481fd...,#красивыедевушки #попа #ass
97,97,97,https://cdn-st.rutubelist.ru/media/27/65/39b47...,#путешествия #journey #туризм #море #красивыем...
98,98,98,https://cdn-st.rutubelist.ru/media/d1/0f/f41e4...,NaN


In [9]:
!pwd

/home/jovyan/anvlasov/hack_yappy/git/notebooks


In [10]:
import sys

sys.path.insert(
    0,
    "/home/jovyan/anvlasov/hack_yappy/git/",
)

In [11]:
import torch

In [12]:
import nest_asyncio
nest_asyncio.apply()

In [13]:
from src.engine.config import ConfigVideoProcessor
from src.engine.model import VideoProcessor

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
device.type

'cuda'

In [16]:
config = ConfigVideoProcessor()
processor = VideoProcessor(config=config, device=device)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### запуск на 1 примере

In [10]:
# !pip install opencv-python

In [19]:
video_url :str = 'https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4'
results = processor.process_video_from_link(video_url, verbose=True)

video_path:data/data_video/_b0_e9_ef285e0241139fc611318ed33071_fhd.mp4
audio_file_path:data/data_audio/_b0_e9_ef285e0241139fc611318ed33071_fhd.mp3
caption:a soccer game with players on the field



Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


transcription: Thank you.
recognition:{'title': 'Resurrection', 'subtitle': 'TapiuanTheGod', 'url': 'https://www.shazam.com/track/621476879/resurrection'}


In [20]:
results

{'captions': 'a soccer game with players on the field\n',
 'transcription': ' Thank you.',
 'shazam title': 'Resurrection',
 'shazam subtitle': 'TapiuanTheGod',
 'shazam url': 'https://www.shazam.com/track/621476879/resurrection'}

### запуск на 10 примерах

In [30]:
results_all = []
for row in tqdm_notebook(df_part.head(10).itertuples()):
    video_url :str = row.link
    result = processor.process_video_from_link(video_url, verbose=False)
    results_all.append(result)

0it [00:00, ?it/s]

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


10/? [01:18<00:00, 7.57s/it]

In [31]:
for index in range(len(results_all)):
    row = df.iloc[index]
    print(index)
    print(f'link:{row.link}')
    print(f'description:{row.description}')
    print(f'results_all:{results_all[index]}')
    print(20*'-')

0
link:https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4
description:#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin
results_all:{'captions': 'a soccer game with players on the field\n', 'transcription': ' Thank you.', 'shazam title': 'Resurrection', 'shazam subtitle': 'TapiuanTheGod', 'shazam url': 'https://www.shazam.com/track/621476879/resurrection'}
--------------------
1
link:https://cdn-st.rutubelist.ru/media/39/6c/b31bc6864bef9d8a96814f1822ca/fhd.mp4
description:🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #roblox #роблокс
results_all:{'captions': 'a cartoon character wearing sunglasses and a white shirt\n', 'transcription': ' Сейчас я тебе покажу секретную команду в Роблоксе. Чтобы её активировать поставь лайк и подпишись, а также введи в чат команду ILOVEYOU. Когда вы её введёте у вас на экране появится вот такой клоун. Скрипт был создан одним из создателей Роблокса и работает только в играх, где есть Роблос дев